# Set references for v8 tuning using the tansig

This notebook is dedicated to apply the linear correction in the neural network output w.r.t the avgmu. 

**NOTE**: This is an extra study.

**NOTE**: Get all models with 5 neurons.

In [1]:
from kolmov import crossval_table, get_color_fader, fit_table
import saphyra
import numpy as np
import pandas as pd
import collections
import os
import matplotlib
import matplotlib.pyplot as plt
from pprint import pprint
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.23/01
Using all sub packages with ROOT dependence


In [2]:
etbins = [15, 20, 30, 40, 50, 1000000]
etabins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]

## 1) Reading all tunings:

Since I don't have v6 tuning files available, I will reload the production files and get the models for each bin.

In [3]:
def get_models( location , etbins, etabins, remove_last=True):

    from tensorflow.keras.models import Model, model_from_json
    from tensorflow.keras.layers import Activation
    import json
    models = [[ None for _ in range(len(etabins)-1)] for __ in range(len(etbins)-1)]
    for et_bin in range(len(etbins)-1):
        for eta_bin in range(len(etabins)-1):
            d_tuned = {}
            path=location.format(ET=et_bin,ETA=3 if eta_bin==4 else eta_bin)
            #print(path)
            with open(path+'.json', 'r') as json_file:
                model = model_from_json(json_file.read())
                # load weights into new model
                model.load_weights(path+".h5")
                # since the production version has no tansig fucntion as default. Let's include it
                # for study
                model.add(Activation('tanh'))
                new_model = Model(model.inputs, model.layers[-2].output) if remove_last else model
                d_tuned['model']    = new_model
                d_tuned['etBin']    = [etbins[et_bin], etbins[et_bin+1]]
                d_tuned['etaBin']   = [etabins[eta_bin], etabins[eta_bin+1]]
                d_tuned['etBinIdx'] = et_bin
                d_tuned['etaBinIdx']= eta_bin
                models[et_bin][eta_bin] = d_tuned
                
    # return the models's matrix          
    return models

model_path = '/Volumes/castor/tuning_data/Zee/production/TrigL2_20180125_v8/models'
model_path+='/data17_13TeV_EGAM1_probes_lhmedium_EGAM7_vetolhvloose.model_v8.electronTight.et{ET}_eta{ETA}'
best_models = get_models(model_path, etbins, etabins,remove_last=True)


In [4]:
best_models[0][0]['model'].summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_input (InputLayer)     [(None, 100)]             0         
_________________________________________________________________
dense (Dense)                (None, 5)                 505       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6         
Total params: 511
Trainable params: 511
Non-trainable params: 0
_________________________________________________________________


## 2) Linear correction:

Here we will set all thresholds to operate as the same pd reference from cut-based using the pileup linear correction strategy. As the classifier efficiency has some "dependence" w.r.t the pileup we adopt the linear adjustment to "fix" the trigger efficiency. Here we will "fix" the neural network threshold w.r.t the pileup. 

### 2.1) Get all PD/Fas values:

Read all reference values from the storage.

In [5]:
# calculate all pd/fa from reference file
ref_path = '/Volumes/castor/cern_data/files/Zee/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97/references/'
ref_path+= 'data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97_et{ET}_eta{ETA}.ref.pic.gz'

ref_paths = [[ ref_path.format(ET=et,ETA=eta) for eta in range(5)] for et in range(5)]
ref_matrix = [[ {} for eta in range(5)] for et in range(5)]
references = ['tight_cutbased', 'medium_cutbased' , 'loose_cutbased', 'vloose_cutbased']
from saphyra.core import ReferenceReader
for et_bin in range(5):
    for eta_bin in range(5):
        for name in references:
            refObj = ReferenceReader().load(ref_paths[et_bin][eta_bin])
            pd = refObj.getSgnPassed(name)/refObj.getSgnTotal(name)
            fa = refObj.getBkgPassed(name)/refObj.getBkgTotal(name)
            ref_matrix[et_bin][eta_bin][name] = {'pd':pd, 'fa':fa}

### 2.2) Create data generator:

Since each tuning models is fed by a different data organization, we need to create a generator to open the data file, prepare the matrix and apply some pre-processing (if needed).

In [6]:
def generator( path ):
    def norm1( data ):
        norms = np.abs( data.sum(axis=1) )
        norms[norms==0] = 1
        return data/norms[:,None]
    from Gaugi import load
    d = load(path)
    feature_names = d['features'].tolist()

    # How many events?
    n = d['data'].shape[0]
    
    # extract rings
    data_rings = norm1(d['data'][:,1:101])
    target = d['target']
    avgmu = d['data'][:,0]
    
    return [data_rings], target, avgmu

In [7]:
path = '/Volumes/castor/cern_data/files/Zee/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97/'
path+= 'data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97_et{ET}_eta{ETA}.npz'
paths = [[ path.format(ET=et,ETA=eta) for eta in range(5)] for et in range(5)]

In [8]:
# create the table class
from ROOT import kBlackBody
ct  = fit_table( generator, etbins , etabins, 0.001, 1.5, 16.5, 60.5, 
                 xmin_percentage=0.0, xmax_percentage=100, palette=kBlackBody )

### 2.3) Apply linear correction:

**NOTE**: Take about 25 minutes.

In [9]:
# Fill it
ct.fill(paths, best_models, ref_matrix,'correction_v8_data17_13TeV_EGAM1_probes_lhmedium_EGAM7_VProbes')


Applying ATLAS style settings...
Fitting... |############################################################| 25/25
Fitting... ... finished task in 2629.850245s.


In [10]:
ct.table().head()

,name,et_bin,eta_bin,reference_signal_passed,reference_signal_total,reference_signal_eff,reference_background_passed,reference_background_total,reference_background_eff,signal_passed,...,signal_eff,background_passed,background_total,background_eff,signal_corrected_passed,signal_corrected_total,signal_corrected_eff,background_corrected_passed,background_corrected_total,background_corrected_eff
0,tight_cutbased,0,0,227619,232819,0.977666,23318,187639,0.124271,227616,...,0.977652,4695,187639,0.025021,227615,232819,0.977648,4541,187639,0.024201
1,medium_cutbased,0,0,227780,232819,0.978360,24336,187639,0.129701,227776,...,0.978339,4761,187639,0.025373,227754,232819,0.978245,4607,187639,0.024552
2,loose_cutbased,0,0,229996,232819,0.987876,31867,187639,0.169837,229994,...,0.987866,6541,187639,0.034859,229981,232819,0.987810,6385,187639,0.034028
3,vloose_cutbased,0,0,230152,232819,0.988548,32748,187639,0.174527,230150,...,0.988536,6767,187639,0.036064,230132,232819,0.988459,6613,187639,0.035243
4,tight_cutbased,0,1,137861,141000,0.977742,31938,143657,0.222321,137861,...,0.977738,5785,143657,0.040270,137855,141000,0.977695,5649,143657,0.039323


### 2.3) Create beamer report:

In [11]:
ct.dump_beamer_table(ct.table(), best_models, 'data17_13TeV v8 tuning', 
                                              'correction_v8_data17_13TeV_EGAM1_probes_lhmedium_EGAM7_VProbes.pdf')

2021-02-15 12:40:18,975 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file correction_v8_data17_13TeV_EGAM1_probes_lhmedium_EGAM7_VProbes.pdf latex code...
